# 原稿の図作成ノートブック

In [96]:
import numpy as np
import matplotlib
matplotlib.use('Agg')  # GUI を使わないバックエンドに設定
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import FancyArrowPatch
from pathlib import Path
import pickle
from config import method_name_dict, DT, V_MIN, V_MAX, W_MIN, W_MAX, A_MAX, AW_MAX


plt.rcParams["font.size"] = 9  # 全体のフォントサイズが変更されます。
plt.rcParams["font.family"] = "Times New Roman"  # 全体のフォントを設定
plt.rcParams["mathtext.fontset"] = "stix"  # math fontの設定
plt.rcParams["font.weight"] = "normal"  # フォントの太さを細字に設定
plt.rcParams["axes.linewidth"] = 1.0  # axis line width
plt.rcParams["axes.grid"] = True  # make grid
plt.rcParams["legend.edgecolor"] = "black"  # edgeの色を変更
plt.rcParams["legend.handlelength"] = 1  # 凡例の線の長さを調節

In [98]:
from path import step_curves
paths_step = step_curves()
path = paths_step[-1]

# pickleを読み込む
results_path = sorted(Path("/home/ytpc2022e/decwest_workspace/dwpp/results").glob("step*"))

# 図1

PP, APP, RPP, DWPPの目標回転速度、実現回転速度を比較した図

一番RMSEの差が大きかった経路7の、回転速度における図を書く


In [112]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_velocities_dict_path = result_path / "robot_velocities_dict.pkl"
with open(robot_velocities_dict_path, "rb") as f:
    robot_velocities_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_ref_velocities_dict_path = result_path / "robot_ref_velocities_dict.pkl"
with open(robot_ref_velocities_dict_path, "rb") as f:
    robot_ref_velocities_dict = pickle.load(f)
# timestampsの読み込み
time_stamp_dict_path = result_path / "time_stamp_dict.pkl"
with open(time_stamp_dict_path, "rb") as f:
    time_stamp_dict = pickle.load(f)

In [140]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5, 4))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

def plot_velocity_profile(ax, robot_velocities, robot_ref_velocities, time_stamps, title, add_legend=False):
    time_stamps = np.array(time_stamps)
    robot_ref_velocities = np.array(robot_ref_velocities)
    robot_velocities = np.array(robot_velocities)

    # 速度の上限・下限線
    ax.axhline(y=W_MAX, linestyle="--", c="black")
    ax.axhline(y=W_MIN, linestyle="--", c="black")
    
    # 凡例のラベルを付けるのは最初のプロットだけ
    if add_legend:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], label='$\omega_{ref}$', c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], label='$\omega$', c="red")
    else:
        ax.plot(time_stamps, robot_ref_velocities[:, 1], c="blue")
        ax.plot(time_stamps, robot_velocities[:, 1], c="red")

    ax.set_title(title)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('angular velocity [rad/s]')
    ax.set_xlim(0, 21.0)
    ax.set_ylim(W_MIN-0.2, W_MAX+0.2)

# 最初のプロット(ax1) のみ凡例ラベルを設定
plot_velocity_profile(ax1, robot_velocities_dict["pp"], robot_ref_velocities_dict["pp"], time_stamp_dict["pp"], "Pure Pursuit", add_legend=True)
plot_velocity_profile(ax2, robot_velocities_dict["app"], robot_ref_velocities_dict["app"], time_stamp_dict["app"], "Adaptive Pure Pursuit")
plot_velocity_profile(ax3, robot_velocities_dict["rpp"], robot_ref_velocities_dict["rpp"], time_stamp_dict["rpp"], "Regulated Pure Pursuit")
plot_velocity_profile(ax4, robot_velocities_dict["dwpp"], robot_ref_velocities_dict["dwpp"], time_stamp_dict["dwpp"], "Dynamic Window Pure Pursuit")

# `ax1` の凡例情報を取得し、fig.legend() に渡す
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.15), ncol=2)

plt.tight_layout()

# グラフを保存
plt.savefig(f'../results/figure1.svg')


# 図4

vω空間におけるRPP, DWPPの目標速度決定の違い

ここはスライドで作る。実現された結果は、図7で示す。

# 図5

経路の外観図を書く

In [141]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(111)
ax.set_xlim(np.min(path[:, 0]) - 1, np.max(path[:, 0]) + 1)
ax.set_ylim(np.min(path[:, 1]) - 1, np.max(path[:, 1]) + 1)
ax.set_xlabel('$x$ [m]')
ax.set_ylabel('$y$ [m]')
ax.set_aspect('equal')

# パス
ax.plot(path[:, 0], path[:, 1], 'k--', label='Path')

plt.tight_layout()

# グラフを表示
plt.savefig(f'../results/path_example.svg')

# 図6

最もRMSEで差がついた参照経路における各種法の追従経路

制約違反したところはハイライトする

## 最もRMSEで差がついた場所

In [20]:
for result_path in results_path:
    print(result_path)
    rmse_path = result_path / "rmse_dict.pkl"
    with open(rmse_path, "rb") as f:
        results = pickle.load(f)
    for key in results:
        print(key, results[key])

/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_0
pp 0.019133561910436584
app 0.02033568924789376
rpp 0.020433731871953593
dwpp 0.020211633003320243
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_1
pp 0.01819043386174176
app 0.019572215834299782
rpp 0.019632124894792657
dwpp 0.019204306983573137
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_2
pp 0.018631241930987846
app 0.01965391343711651
rpp 0.01961759517473724
dwpp 0.019420072921760533
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_3
pp 0.11647772816323401
app 0.10054840484383912
rpp 0.03392521301690285
dwpp 0.03464689233143726
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_4
pp 0.15272137618747889
app 0.1030939018709039
rpp 0.03285514120437314
dwpp 0.03287012998713747
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_5
pp 0.14448296961894402
app 0.09909577871802867
rpp 0.0317456881457999
dwpp 0.031240790851314607
/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve

一番RMSEが小さいのは、経路7

In [14]:
results_path

[PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_0'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_1'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_2'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_3'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_4'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_5'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_6'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_7'),
 PosixPath('/home/ytpc2022e/decwest_workspace/dwpp/results/step_curve_8')]

In [142]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111)
ax.set_xlim(np.min(path[:, 0]) - 0.1, np.max(path[:, 0]) + 0.1)
ax.set_ylim(np.min(path[:, 1]) - 0.1, np.max(path[:, 1]) + 1)
ax.set_xlabel('$x$ [m]')
ax.set_ylabel('$y$ [m]')
ax.set_aspect('equal')

# パス
ax.plot(path[:, 0], path[:, 1], 'k--', label='Reference Path')


# 制約違反した場所をハイライト表示
for method_name, break_constraints_flags in break_constraints_flags_dict.items():
    break_constraints_flags = np.array(break_constraints_flags)
    
    for i, flag in enumerate(break_constraints_flags):
        if flag[0] and flag[1]: # 並進回転共に違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=5,
                linestyle='None',     # 線でつなげない
                color='yellow',
                alpha=0.5             # 半透明（0〜1の値が指定可）
            )
        elif flag[0]: # 並進だけ違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=5,
                linestyle='None',     # 線でつなげない
                color='red',
                alpha=0.5             # 半透明（0〜1の値が指定可）
            )
        elif flag[1]: # 回転だけ違反
            x = robot_poses_dist[method_name][i][0]
            y = robot_poses_dist[method_name][i][1]
            ax.plot(
                x,
                y,
                marker='o',
                markersize=5,
                linestyle='None',     # 線でつなげない
                color='blue',
                alpha=0.5             # 半透明（0〜1の値が指定可）
            )


# ロボットの位置
for method_name, robot_poses in robot_poses_dist.items():
    robot_poses = np.array(robot_poses)
    ax.plot(robot_poses[:, 0], robot_poses[:, 1], label=method_name_dict[method_name])

# 凡例を表示
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)

plt.tight_layout()

# グラフを表示
plt.savefig(f'../results/tracking_result_example.svg')

# 図7

最もRMSEで差がついたl=3.0, θ=135°の参照経路におけるRPP, DWPP with RPPのvω空間におけるDynamic windowと目標速度の遷移を図7に図示する。 

- 速度プロファイル
  - 走行位置のズレが少ない曲がり始めに着目してプロットする（完了）

Dynamic Window付近を切り取って可視化するようにする。

- vreg>=vminの好例（並進速度が減速することを示す）
  - 経路7のRPP, DWPPの31番目を同時にプロットする！！
- vreg<=vminの好例（回転速度が増加することを示す）
  - DWPPの36番目と、RPPの47番目あたりはいい対比になる。

速度プロファイルを比較する

In [58]:
path = paths_step[7]

# pickleを読み込む
result_path = results_path[7]
# robot posesの読み込み
robot_poses_path = result_path / "robot_poses_dict.pkl"
with open(robot_poses_path, "rb") as f:
    robot_poses_dist = pickle.load(f)
# break_constraints_flagsの読み込み
break_constraints_flags_path = result_path / "break_constraints_flags_dict.pkl"
with open(break_constraints_flags_path, "rb") as f:
    break_constraints_flags_dict = pickle.load(f)
# robot_velocities_dictの読み込み
robot_velocities_dict_path = result_path / "robot_velocities_dict.pkl"
with open(robot_velocities_dict_path, "rb") as f:
    robot_velocities_dict = pickle.load(f)
# timestampsの読み込み
time_stamp_dict_path = result_path / "time_stamp_dict.pkl"
with open(time_stamp_dict_path, "rb") as f:
    time_stamp_dict = pickle.load(f)

並進速度

In [80]:
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)
ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax.axhline(y=V_MAX, linestyle="--", c="black")
ax.axhline(y=V_MIN, linestyle="--", c="black")

rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

# 並進目標速度
ax.plot(rpp_timestamp, rpp_velocities[:, 0], c="blue", label='RPP')
# 並進実現速度
ax.plot(dwpp_timestamp, dwpp_velocities[:, 0], c="red", label='DWPP')

# x軸の値の範囲を指定
ax.set_xlim(0, 23.0)

# 凡例を表示
plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.15), ncol=2)

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7_trans.png')


回転速度

In [81]:
fig = plt.figure(figsize=(4, 2))
ax = fig.add_subplot(111)
ax.set_xlabel('$t$ [s]')
ax.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax.axhline(y=W_MAX, linestyle="--", c="black")
ax.axhline(y=W_MIN, linestyle="--", c="black")

rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

# 並進目標速度
ax.plot(rpp_timestamp, rpp_velocities[:, 1], c="blue", label='RPP')
# 並進実現速度
ax.plot(dwpp_timestamp, dwpp_velocities[:, 1], c="red", label='DWPP')

# x軸の値の範囲を指定
ax.set_xlim(0, 23.0)

# 凡例を表示
plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.15), ncol=2)

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7_rot.png')


並進・回転を一枚で書く

In [91]:
rpp_timestamp = np.array(time_stamp_dict["rpp"])
rpp_velocities = np.array(robot_velocities_dict["rpp"])
dwpp_timestamp = np.array(time_stamp_dict["dwpp"])
dwpp_velocities = np.array(robot_velocities_dict["dwpp"])

fig = plt.figure(figsize=(4, 3))
ax1 = fig.add_subplot(211)
ax1.set_xlabel('$t$ [s]')
ax1.set_ylabel('$v$ [m/s]')
# 速度の上限・下限線
ax1.axhline(y=V_MAX, linestyle="--", c="black")
ax1.axhline(y=V_MIN, linestyle="--", c="black")

# 並進目標速度
ax1.plot(rpp_timestamp, rpp_velocities[:, 0], c="blue", label='RPP')
# 並進実現速度
ax1.plot(dwpp_timestamp, dwpp_velocities[:, 0], c="red", label='DWPP')

# x軸の値の範囲を指定
ax1.set_xlim(0, 3.0)

ax2 = fig.add_subplot(212)
ax2.set_xlabel('$t$ [s]')
ax2.set_ylabel('$\omega$ [m/s]')
# 速度の上限・下限線
ax2.axhline(y=W_MAX, linestyle="--", c="black")
ax2.axhline(y=W_MIN, linestyle="--", c="black")

# 並進目標速度
ax2.plot(rpp_timestamp, rpp_velocities[:, 1], c="blue", label='RPP')
# 並進実現速度
ax2.plot(dwpp_timestamp, dwpp_velocities[:, 1], c="red", label='DWPP')

# x軸の値の範囲を指定
ax2.set_xlim(0, 3.0)

# 凡例を表示
plt.legend(loc='upper right', bbox_to_anchor=(0.4, -0.3), ncol=2)

# ax1.legend()
plt.tight_layout()


# グラフを表示
plt.savefig(f'../results/figure7.png')
